In [317]:
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials
import csv
import os
import pandas as pd
import numpy as np 
from collections import OrderedDict

In [318]:
#Authentication - without user
cid = "ENTER HERE"
secret = "ENTER HERE"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [319]:
#Import tracks from .csv
bbcsv = pd.read_csv('billboard_top_songs.csv')

In [320]:
df2 = bbcsv[['title', 'artist']].copy()

In [321]:
df2.replace('\'','', regex=True, inplace=True)
df2.replace('\"','', regex=True, inplace=True)
#df2.replace('','', regex=True, inplace=True)
#df2.replace('[^A-Za-z\s]','', regex=True, inplace=True)

In [312]:
sample = df2.head(200)

In [269]:
#Get Track ID

artist = sample['artist'][22]
track = sample['title'][22]

track_id = sp.search(q='artist:' + artist + ' track:' + track, type='track', market = 'US')

In [270]:
track_id

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=artist%3ATeena+Marie+track%3ALover+Girl&type=track&market=US&offset=0&limit=10',
  'items': [],
  'limit': 10,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0}}

In [205]:
items = track_id['tracks']['items']

In [243]:
track = items[0]

IndexError: list index out of range

In [201]:
song_id = str(track["id"])
track_features=sp.audio_features(song_id)

In [330]:
type(track_features[0])

dict

In [126]:
features_to_df = np.array(track_features)[0]
features_to_df = OrderedDict(features_to_df)

In [128]:
#Get Dictionary values
feature_values = features_to_df.values()

odict_values([0.771, 0.655, 3, -10.842, 0, 0.0372, 0.152, 0.00197, 0.0685, 0.97, 119.747, 'audio_features', '1ZPlNanZsJSPK5h9YZZFbZ', 'spotify:track:1ZPlNanZsJSPK5h9YZZFbZ', 'https://api.spotify.com/v1/tracks/1ZPlNanZsJSPK5h9YZZFbZ', 'https://api.spotify.com/v1/audio-analysis/1ZPlNanZsJSPK5h9YZZFbZ', 218627, 4])

In [331]:
def get_spotify_features(track, artist):
    #Search for Spofity song ID 
    songs=sp.search(q='track:'+track+' '+'artist:'+artist+'*' , type='track', market = 'US')
    items = songs['tracks']['items']
    if len(items) ==0:
        return([0]*len(feature_values))
    else:
        track = items[0]
        song_id = str(track["id"])
        #Use ID to get Song features
        track_features=sp.audio_features(song_id)
        #if len(track_features[0]) <18:
        #    return([0]*len(feature_values))
        if len(track_features[0]) = None:
            continue 
        else:
            features_to_df = np.array(track_features)[0]
            #Order Dictionary
            features_to_df = OrderedDict(features_to_df)
            #Get Dictionary values
            feature_values = features_to_df.values()
            return(feature_values)

In [332]:
dfx = df2.apply(lambda x: get_spotify_features(x['title'], x['artist']), axis=1)

TypeError: 'NoneType' object is not iterable

In [315]:
df1 = pd.DataFrame(dfx.tolist())
df1.columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature']

In [316]:
sample.join(df1)

,title,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Like A Virgin,Madonna,0.771,0.655,3.0,-10.842,0.0,0.0372,0.15200,0.001970,0.0685,0.970,119.747,audio_features,1ZPlNanZsJSPK5h9YZZFbZ,spotify:track:1ZPlNanZsJSPK5h9YZZFbZ,https://api.spotify.com/v1/tracks/1ZPlNanZsJSP...,https://api.spotify.com/v1/audio-analysis/1ZPl...,218627.0,4.0
1,I Want To Know What Love Is,Foreigner,0.477,0.471,6.0,-10.036,1.0,0.0277,0.19300,0.000002,0.1310,0.423,81.204,audio_features,1JLn8RhQzHz3qDqsChcmBl,spotify:track:1JLn8RhQzHz3qDqsChcmBl,https://api.spotify.com/v1/tracks/1JLn8RhQzHz3...,https://api.spotify.com/v1/audio-analysis/1JLn...,304787.0,4.0
2,I Feel For You,Chaka Khan,0.698,0.912,11.0,-8.993,1.0,0.0675,0.01450,0.022500,0.2790,0.797,124.917,audio_features,5yR9u8QiOt8hJaddv32oo7,spotify:track:5yR9u8QiOt8hJaddv32oo7,https://api.spotify.com/v1/tracks/5yR9u8QiOt8h...,https://api.spotify.com/v1/audio-analysis/5yR9...,343400.0,4.0
3,Out Of Touch,Daryl Hall and John Oates,0.806,0.649,9.0,-9.004,0.0,0.0347,0.03910,0.000056,0.0557,0.897,114.834,audio_features,7o67roCVsFiCt7Cf0ZLOJq,spotify:track:7o67roCVsFiCt7Cf0ZLOJq,https://api.spotify.com/v1/tracks/7o67roCVsFiC...,https://api.spotify.com/v1/audio-analysis/7o67...,249573.0,4.0
4,Everybody Wants To Rule The World,Tears For Fears,0.645,0.795,7.0,-12.095,1.0,0.0527,0.34700,0.003890,0.1040,0.535,112.067,audio_features,4RvWPyQ5RL0ao9LPZeSouE,spotify:track:4RvWPyQ5RL0ao9LPZeSouE,https://api.spotify.com/v1/tracks/4RvWPyQ5RL0a...,https://api.spotify.com/v1/audio-analysis/4RvW...,251489.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,Bad,Michael Jackson,0.787,0.889,8.0,-3.786,1.0,0.0397,0.00462,0.423000,0.0665,0.394,114.091,audio_features,5lWFrW5T3JtxVCLDb7etPu,spotify:track:5lWFrW5T3JtxVCLDb7etPu,https://api.spotify.com/v1/tracks/5lWFrW5T3Jtx...,https://api.spotify.com/v1/audio-analysis/5lWF...,247360.0,4.0
196,Change Of Heart,Cyndi Lauper,0.706,0.847,1.0,-6.134,1.0,0.0831,0.22600,0.000104,0.0632,0.495,119.768,audio_features,4Kvz5bhHNYPSaGnQ7PsTBi,spotify:track:4Kvz5bhHNYPSaGnQ7PsTBi,https://api.spotify.com/v1/tracks/4Kvz5bhHNYPS...,https://api.spotify.com/v1/audio-analysis/4Kvz...,264773.0,4.0
197,Cant We Try,Dan Hill,0.561,0.319,9.0,-13.418,0.0,0.0313,0.46600,0.000000,0.0684,0.329,120.106,audio_features,0x1RspO36DgmXbZ9XmxvsN,spotify:track:0x1RspO36DgmXbZ9XmxvsN,https://api.spotify.com/v1/tracks/0x1RspO36Dgm...,https://api.spotify.com/v1/audio-analysis/0x1R...,239667.0,4.0
198,To Be A Lover,Billy Idol,0.562,0.767,5.0,-8.025,1.0,0.0474,0.23400,0.008170,0.0828,0.526,184.363,audio_features,4nBzre6D0UrQFj1s8z45e1,spotify:track:4nBzre6D0UrQFj1s8z45e1,https://api.spotify.com/v1/tracks/4nBzre6D0UrQ...,https://api.spotify.com/v1/audio-analysis/4nBz...,232387.0,4.0
